# Walk around the Fort - Data Set Resolution Reduction - Horizontal and Vertical

In this notebook, I will explore various methods for reducing the resolution of a dataset, both horizontally (reducing the number of observations) and vertically (reducing the number of different observed values). The dataset under scrutiny is the Strava recording of a 5 km walk. 

Strava collects data from athletes regarding their activities - such as running, cycling, walking and hiking. Members can upload data - and tens of millions do so, including some well known cyclists. I have recorded a walk around the fort with a colleague as a Strava activity.

All details for this activity can be retrieved using: https://www.strava.com/activities/2693790029/streams?stream_types%5B%5D=time&stream_types%5B%5D=velocity_smooth&stream_types%5B%5D=watts_calc&stream_types%5B%5D=altitude&stream_types%5B%5D=heartrate&stream_types%5B%5D=cadence&stream_types%5B%5D=temp&stream_types%5B%5D=distance&stream_types%5B%5D=latlng&stream_types%5B%5D=grade_smooth&_=1565273406470

Note: I do not know yet the meaning or even relevance of the last parameter.
Note 2: these URLs can only be accessed from an authenticated session (authenticated in a browser with a valid Strava account).

I have extracted the data and stored in the file walk-around-the-fort.json.


# Read Data into Pandas
Let's load the JSON data from file into a Pandas Data Frame - just like we almost always do with data to analyze in a Python based Jupyter Notebook. Then we can inspect, wrangle and explore the data set and start preparing for visualization and further processing.

In [43]:
import pandas as pd
import json

def readData(filename):
    s = pd.read_json(filename)
    # calculate speed in km/h
    s['speed']= s['velocity_smooth']*3.6
    s['lat']=s['latlng'].apply(lambda ll: ll[0])
    s['long']=s['latlng'].apply(lambda ll: ll[1])
    return s

s = readData("walk-around-the-fort.json")
s.head(10)

,altitude,latlng,velocity_smooth,grade_smooth,distance,time,speed,lat,long
0,2.6,"[52.033722, 5.09888]",0.0,2.3,0.0,0,0.00,52.033722,5.098880
1,2.8,"[52.033854, 5.098836]",1.2,1.8,8.7,7,4.32,52.033854,5.098836
2,2.9,"[52.033887, 5.098811]",1.3,1.4,12.8,10,4.68,52.033887,5.098811
3,2.9,"[52.033916, 5.09878]",1.3,1.2,16.7,13,4.68,52.033916,5.098780
4,2.9,"[52.033953, 5.098756]",1.4,0.6,21.1,16,5.04,52.033953,5.098756
5,3.0,"[52.033987, 5.09875]",1.4,0.6,24.9,19,5.04,52.033987,5.098750
6,3.0,"[52.034021, 5.098724]",1.1,1.3,29.1,23,3.96,52.034021,5.098724
7,3.0,"[52.034048, 5.098727]",1.2,0.6,32.1,25,4.32,52.034048,5.098727
8,3.1,"[52.034084, 5.098739]",1.4,0.7,36.3,28,5.04,52.034084,5.098739
9,3.1,"[52.034125, 5.09874]",1.2,0.6,40.8,32,4.32,52.034125,5.098740


In [93]:
# the number of data points:
s.shape[0]

1325

In [150]:
# visualization of the altitude measurement for all 1300+ data points (suggesting an accuray that is not realistic) 
fig = px.line(s, x="time", y="altitude", title='Altitude vs Distance in our Walk Around the Fort', render_mode='svg')
fig.show()

# Horizontal Resolution Reduction
Instead of 1300+ data points, measurements for every few meters, let's bring that number down by taking just one record for every 25 meters.

Let's take an average altitude per 25 meters. First, assign each observation to a 25 meter bucket, then group by bucket while calculating altitude average.  

In [134]:
distance_window_width = 25 
s['distance_window'] = s['distance'].apply(lambda distance: distance_window_width*(round(distance/distance_window_width)))
s[['altitude','distance_window']].head(10)

,altitude,distance_window
0,2.6,0
1,2.8,0
2,2.9,25
3,2.9,25
4,2.9,25
5,3.0,25
6,3.0,25
7,3.0,25
8,3.1,25
9,3.1,50


In [137]:
d = s[['altitude','distance_window']].copy().groupby(s['distance_window']).mean()
print("Number of records in data frame ",d.shape[0])
d.head(10)

Number of records in data frame  200


,altitude,distance_window
distance_window,,
0,2.700000,0
25,2.971429,25
50,3.150000,50
75,3.233333,75
100,3.557143,100
125,4.042857,125
150,4.450000,150
175,4.500000,175
200,4.583333,200


In [152]:
fig = px.bar(d, x="distance_window", y="altitude", title='Altitude vs Distance in our Walk Around the Fort - average per distance window')
fig.show()

## PAA - Piecewise Aggregate Approximation
Instead of the somewhat naive, crude approach taken in the previous section, we can make use of PAA or piecewise aggregate approximation to reduce the resolution in horizontal direction. PAA also calculates the mean value of our signal for equi width windows, but does so in a slightly more subtle way. For more details, read for example [this blog article](http://vigne.sh/posts/piecewise-aggregate-approx/). 

In [89]:
# this function returns an array of <paa_segments> agregate values, calculates as aggregate aproximations over the values from <series> 
# taken from https://github.com/seninp/saxpy/blob/master/saxpy/paa.py
def paa(series, paa_segments):
    """PAA implementation."""
    series_len = len(series)

    # check for the trivial case
    if (series_len == paa_segments):
        return np.copy(series)
    else:
        res = np.zeros(paa_segments)
        # check when we are even
        if (series_len % paa_segments == 0):
            inc = series_len // paa_segments
            for i in range(0, series_len):
                idx = i // inc
                np.add.at(res, idx, series[i])
                # res[idx] = res[idx] + series[i]
            return res / inc
        # and process when we are odd
        else:
            for i in range(0, paa_segments * series_len):
                idx = i // series_len
                pos = i // paa_segments
                np.add.at(res, idx, series[pos])
                # res[idx] = res[idx] + series[pos]
            return res / series_len

In [151]:
# to bring down the number of data points from 1300 to for example 130, use the PAA algorithm like this:
e = paa(series = s['altitude'], paa_segments = 130) 
e

array([2.93320755, 3.19584906, 3.60867925, 4.32792453, 4.51924528,
       4.54113208, 4.40339623, 4.01698113, 3.35811321, 3.74301887,
       3.85886792, 3.45886792, 2.56641509, 2.41471698, 2.4       ,
       2.43018868, 2.45811321, 2.36603774, 2.2445283 , 2.2       ,
       2.16037736, 2.07811321, 2.        , 1.92528302, 1.9       ,
       1.9       , 1.94339623, 1.21660377, 0.90528302, 0.79207547,
       0.59396226, 1.40150943, 1.65584906, 1.8       , 1.63396226,
       2.49584906, 3.71056604, 3.42981132, 3.79320755, 3.8354717 ,
       3.01396226, 2.47962264, 2.4       , 2.4       , 2.47509434,
       2.41320755, 2.32075472, 2.2       , 2.2       , 2.10943396,
       2.03924528, 1.99018868, 1.9       , 1.9       , 1.90566038,
       1.9109434 , 1.19056604, 0.88339623, 0.80150943, 0.66792453,
       1.51320755, 1.73584906, 1.8       , 1.54490566, 3.37962264,
       3.45207547, 3.5690566 , 3.82037736, 3.70754717, 2.81245283,
       2.46415094, 2.4       , 2.4       , 2.46113208, 2.43698

In [149]:
# create Pandas data frame from  numpy.ndarray
de = pd.DataFrame(data=e[:],    # values
              index=e[:],    # 1st column as index
              columns=['altitude']
                 ) 
# add an column x that has its values set to the row index of each row
de['x'] = range(1, len(de) + 1)
fig = px.bar(de, x="x", y="altitude", title='PAA outcome: Altitude vs Distance window in our Walk Around the Fort')
fig.show()

# Vertical Data Resolution Reduction
In addition to lowering the number of observations, we can also do some vertical resolution reduction. Instead of pretending we have meaningfully different altitude measurements with 7 significant digits - and we need all of those digits - we can simplify and honestize our data set. GPS does not measure altitude very accurately. And for our purposes, we only need to see differences and fluctuations, not absolute values. 

To let go of make believe accuracy and stick to the essence we are after will make our data easier to interpret, smaller to hold in memory and faster to process. Win, win, win. It may feel as though we are losing potentially relevant data, but we are not. First because the suggested accuracy is fake. And even if it were real - we do not need it. So it would only be in our way.

We can for example work with a small number of levels or even categories and assign each altitude value to one of these levels. Small differences in measurement (probably meaningless anyway) are smoothed out, while the overall signal that we are interested in is preserved.

One method is binning - with equal height bins. We divide the altitude value range - roughly 0.5 to 4.5 meters over sea level - in for example six categories and then determine for each measured altitude value in which of the six bins it belongs. Thus we assign bin numbers of labels. This is done in the next section: 

In [145]:
# distribute altitude over limited number of equal height buckets
# note: the pd.cut function also allows us to specify our own bin value ranges as well as our own bin labels
number_of_bins = 6
d['altitude_bin'] = pd.cut(d['altitude'], number_of_bins,labels=False)
d.head(10)

,altitude,distance_window,altitude_bin,x
distance_window,,,,
0,2.700000,0,3,0
25,2.971429,25,3,25
50,3.150000,50,3,50
75,3.233333,75,4,75
100,3.557143,100,4,100
125,4.042857,125,5,125
150,4.450000,150,5,150
175,4.500000,175,5,175
200,4.583333,200,5,200


In [148]:
# here we get the boundaries of the bins and the number of measurements assigned to each bin. The highest number in the most average bin
d['altitude_bin'] = pd.cut(d['altitude'], number_of_bins)
d['altitude_bin'].value_counts(sort=False)

(0.424, 1.121]    20
(1.121, 1.814]    30
(1.814, 2.507]    90
(2.507, 3.2]      14
(3.2, 3.893]      29
(3.893, 4.586]    17
Name: altitude_bin, dtype: int64

Visualizing the bin assignments for each distance window. The signal is still there - but with far less detail. Can we still count the number of laps we walked around the fort? Has it become easier even to count the number of laps? 

In [146]:
d['x']=d.index
fig = px.bar(d, x="x", y="altitude_bin", title='Altitude Bin Assignment vs Distance Window in our Walk Around the Fort')
fig.show()

Instead of working with bins that each have the same height (cover the same altitude range) we can also work with bins that each have the same number of observations (and varying value ranges). Such bins are called quantiles. 

Below we see the function qcut at work: it assigns quantiles to each altitude value. Here we determine for each observation in which of six quantiles it belongs (highest 1/6 of the population, lowest 1/6 of somewhere in between). You can see both the boundaries for the quantiles as well as the number of observations in each quantile. 

In [154]:
d['altitude_bin'] = pd.qcut(d['altitude'], number_of_bins)
d['altitude_bin'].value_counts(sort=False)

(0.428, 1.6]      34
(1.6, 1.915]      33
(1.915, 2.2]      34
(2.2, 2.479]      32
(2.479, 3.543]    33
(3.543, 4.586]    34
Name: altitude_bin, dtype: int64

We cannot use the assigned interval for our observations in a plotly chart. So instead I assign the boundary of each quantile to the observation. Alternatively, I could have worked with a label for each quantile, as I did with the equiheight bins.

In [156]:
number_of_vertical_bins = 6
de['altitude_bin'] = pd.qcut(de['altitude'], number_of_vertical_bins)
categories, edges = pd.qcut(de['altitude'], number_of_vertical_bins, retbins=True, labels=False)
de2 = pd.DataFrame({'original_altitude':de['altitude'],
                   'altitude_bin': edges[1:][categories]},
                  columns = ['original_altitude', 'altitude_bin'])
de2['x'] = range(1, len(de2) + 1)
de2.head(10)

,original_altitude,altitude_bin,x
2.933208,2.933208,3.513962,1
3.195849,3.195849,3.513962,2
3.608679,3.608679,4.568679,3
4.327925,4.327925,4.568679,4
4.519245,4.519245,4.568679,5
4.541132,4.541132,4.568679,6
4.403396,4.403396,4.568679,7
4.016981,4.016981,4.568679,8
3.358113,3.358113,3.513962,9
3.743019,3.743019,4.568679,10


In [125]:
# show barchart for altitude vs. PAA segment
fig = px.bar(de2, x="x", y="altitude_bin", title='Altitude Quantile vs Distance Window in our Walk Around the Fort')
fig.show()

## SAX - Symbolic Aggregate approXimation

The bin labels in the previous section may appear like measurements with being numeric and all. But in fact they are unit less labels. They could have been labeled A through F. They are ordered but have no size associated with them.

The concept of symbolic representation of time series (and using that compact representation for efficient similarity analysis) has been researched extensively. The most prominent theory in this field to date is called SAX - Symbolic Aggregate approXimation. It also assigns labels to each observed value - going about it in a slightly more subtle way than using equiheight bins or quantiles. Check out one of the many resources on SAX - for example starting from here: https://www.cs.ucr.edu/~eamonn/SAX.htm. 

To create a SAX representation of our walk around the fort is not very hard at all.


In [158]:
# first install library saxpy
import sys
!{sys.executable} -m pip install saxpy
# next import libraries
from saxpy.alphabet import cuts_for_asize
import numpy as np
from saxpy.znorm import znorm
from saxpy.sax import ts_to_string
from saxpy.paa import paa

With the library installed, I can now define the size of the alphabet (how many categories do I want to use for the value range).

In [164]:
# how many different categories to use or how many letters in the SAX alphabet
alphabet_size = 7
# normalize the altitude data series
data_znorm = znorm(s['altitude'])
# use PAA for horizontal resolution reduction from 1300+ data points to 130 segments 
# Note: this is a fairly slow step
data_paa = paa(data_znorm, 130)
# create the SAX representation for the 130 data points
sax_representation_altitude_series = ts_to_string(data_paa, cuts_for_asize(alphabet_size))
sax_representation_altitude_series

'efggggggfgggdddddddccccccccaaaabbbbdgfggfddddddcccccccccaaaabbbbfggggedddddddccccccccaaaabbbbgfggfdddddddccccccccaaaabbbbggggggfed'

I can see an interesting repetition going on in that string. It seems like the substring "ddddddccccccccaaaabbb" is repeated four times (which could correspond with our four laps around the fort).

In [173]:
import numpy as np
sax_array = np.array(list(sax_representation_altitude_series))
# create Pandas data frame from  numpy.ndarray
de = pd.DataFrame(data=sax_array[:],    # values
              index=sax_array[:],    # 1st column as index
              columns=['sax_altitude']
                 ) 
# add an column x that has its values set to the row index of each row
de['sax_category'] = de['sax_altitude'].apply(lambda letter: alphabet_size - (ord(letter)-97))
de['x'] = range(1, len(de) + 1)

In [178]:
fig = px.bar(de, x="x", y="sax_category", title='SAX Altitude Representation in our Walk Around the Fort')

fig.update_layout(
    yaxis = dict(
            ticktext = ['','g', 'f','e', 'd','c', 'b','a'],
            tickvals = [ 0, 1, 2, 3, 4, 5,6,7 ]
    ))
fig.show()